In [1]:
import rasterio

In [36]:
full_tif = '../data/all_data/2023-02-23_Bakonyszucs_actual.tif'
tif_file_path = '../data/2024-04-21-dataset/standard/original/tifs/tile_tif_100_26.tif'

In [7]:
with rasterio.open(tif_file_path) as dataset:
    pixel_width, pixel_height = dataset.res

pixel_area = pixel_width * pixel_height

print(f"Pixel size: {pixel_width} x {pixel_height}")
print(f"Pixel area: {pixel_area}")

Pixel size: 0.019999999999997724 x 0.019999999999999504
Pixel area: 0.00039999999999994456


In [13]:
tif = rasterio.open(tif_file_path)
tif.res

(0.019999999999997724, 0.019999999999999504)

In [70]:
import rasterio
from rasterio.enums import Resampling

def ResampleTIF(upscale_factor, tif_path, resampled_tif_path):
    tif_image = rasterio.open(tif_path)
    # resample
    data = tif_image.read(
        out_shape=(
            tif_image.count,
            int(tif_image.height * upscale_factor),
            int(tif_image.width * upscale_factor)
        ),
        resampling=Resampling.bilinear
    )
    
    #pixel_width, pixel_height = tif_image.res
    #print(f"Transformed pixel size: {pixel_width} x {pixel_height}")
    
    # scale image transform
    new_transform = tif_image.transform * tif_image.transform.scale(
        (tif_image.width / data.shape[-1]),
        (tif_image.height / data.shape[-2])
    )
    #pixel_width = transform.a
    #pixel_height = -transform.e
    
    #print(f"Transformed pixel size: {pixel_width} x {pixel_height}")
    #pixel_area = pixel_width * pixel_height
    #print(tif_image.transform,data.shape,transform)

    profile = tif_image.profile
    profile.update({
        'height': data.shape[1],
        'width': data.shape[2],
        'transform': new_transform
    })
    rasterio.open(resampled_tif_path, 'w', **profile).write(data)

def check_resolution(desired_res, tif_path, round_accuracy):
    '''
    Projected Coordinate System (PCS): the units are typically in meters or feet.
    Geographic Coordinate System (GCS): the units are typically in degrees.
    The CRS (Coordinate Reference System) EPSG:23700 corresponds to the "HD72 / EOV" projection.
    This is a projected coordinate system used in Hungary, and the units for this CRS are meters.
    '''
    tif_image = rasterio.open(tif_path)
    p_width, p_height = tif_image.res
    p_width_normalized = round(p_width,round_accuracy)
    p_height_normalized = round(p_height,round_accuracy)
    #print(p_width_normalized,p_height_normalized)
    
    actual_res = p_width_normalized
    ratio = actual_res/desired_res
    return ratio

In [71]:
ResampleTIF(
    2,
    '../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69.tif',
    '../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69_RESAMPLED.tif'
)

In [61]:
check_resolution(
    0.01999999999999,
    '../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69_RESAMPLED.tif',#../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69.tif',
    10
)

0.50000000000025

In [78]:
tif_image = rasterio.open('../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69.tif')
p_width, p_height = tif_image.res
tif_image.crs

CRS.from_epsg(23700)

In [51]:
p_width

0.019999999999997724

In [77]:
des_ratio = check_resolution(
        0.01999999999999,
        '../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69_RESAMPLED.tif',#../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69.tif',
        10
    ),
print(des_ratio[0])
ResampleTIF(
    des_ratio[0],
    '../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69_RESAMPLED.tif',
    '../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69_ORIG.tif'
)

5.88235294000294


In [76]:
ResampleTIF(
    0.1713,
    '../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69_ORIG.tif',
    '../data/2024-04-21-dataset/standard/original/tifs/tile_tif_0_69_RESAMPLED.tif'
)

In [1]:
import torch

In [2]:
torch.cuda.is_available()

False